In [1]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
import numpy as np

output_notebook()

Loading BokehJS ...

In [2]:
fig = figure(plot_width=400, plot_height=400, toolbar_location='above')

In [3]:
x = np.linspace(0,2*np.pi, 200)
y = np.linspace(0,2*np.pi, 200)
X, Y = np.meshgrid(x,y)
Z = (X-np.pi)**2*np.sin(8*X)-(Y-np.pi)**2*np.cos(8*Y)

im = fig.image(image=[Z], x=0, y=0, dw=10, dh=10, palette='Viridis256')
fig.x_range.start = 0
fig.x_range.end = 10
fig.y_range.start = 0
fig.y_range.end = 10

In [4]:
from bokeh.models.annotations import ColorBar

cm = fig.renderers[-1].glyph.color_mapper # color_mapper of the image
cb = ColorBar(color_mapper=cm, location=(0,0))
fig.add_layout(cb, 'right')

In [5]:
data = im.data_source.data['image'][0]
datamin = data.min() # get the limits of the sliders
datamax = data.max()

In [6]:
from bokeh.models import Slider

# I need to get the values of datamin and datamax into the CustomJS callbacks,
# but CustomJS only accepts Model objects as arguments, so I'm hiding these
# values inside an empty Slider
model = Slider() # trick it into letting datamin and datamax into CustomJS
model.tags.append(datamin) # Hide these in here
model.tags.append(datamax)

In [7]:
# Javascript callbacks, very poorly written and poorly commented, but they work.
# These are for the "upper bound" slider (u), "lower bound" slider (l),
# "upper bound" textbox (ut), "lower bound" textbox (lt),
# and a reset button (reset_js)

from bokeh.models import CustomJS

callback_u = CustomJS(args=dict(cb=cb, im=im, model=model), code="""
        var cm = cb.color_mapper;
        var upp = upper_slider.get('value');
        upper_input.value = upp.toString()
        lower_slider.end = upp
        cm.high = upp;
        im.glyph.color_mapper.high = upp;
        if (cm.low >= cm.high){
        cm.low = upp/1.1 // to prevent limits being the same
        im.glyph.color_mapper.low = low/1.1;
        }
        if (upp > model.tags[1]){ // model.tags[1] is datamax
            upper_slider.end = upp
        }
        """)

callback_l = CustomJS(args=dict(cb=cb, im=im, model=model), code="""
    var cm = cb.color_mapper;
    var low = lower_slider.get('value');
    lower_input.value = low.toString()
    upper_slider.start = low
    cm.low = low;
    im.glyph.color_mapper.low = low;
    if (cm.high <=  cm.low){
    cm.high = low*1.1 // to prevent limits being the same
    im.glyph.color_mapper.high = low*1.1;
    }
    if (low < model.tags[0]){ // model.tags[0] is datamin
        lower_slider.start = low
    }""")

callback_ut = CustomJS(args=dict(cb=cb, im=im, model=model), code="""
    var cm = cb.color_mapper;
    var upp = parseFloat(upper_input.get('value'));
    upper_slider.value = upp
    cm.high = upp;
    im.glyph.color_mapper.high = upp;
    if (cm.low >=  cm.high){
    cm.low = upp/1.1 // to prevent limits being the same
    im.glyph.color_mapper.low = upp/1.1;
    }
    if (upp > model.tags[1]){ // model.tags[1] is datamax
        upper_slider.end = upp
    }
    """)

callback_lt = CustomJS(args=dict(cb=cb, im=im, model=model), code="""
    var cm = cb.color_mapper;
    var low = parseFloat(lower_input.get('value'));
    lower_slider.value = low
    cm.low = low;
    im.glyph.color_mapper.low = low;
    if (cm.high <=  cm.low){
    cm.high = low*1.1 // to prevent limits being the same
    im.glyph.color_mapper.high = low*1.1;
    }
    if (low < model.tags[0]){ // model.tags[0] is datamin
        lower_slider.start = low
    }
    """)

callback_reset_js = CustomJS(args=dict(cb=cb, im=im, model=model), code="""
    var cm = cb.color_mapper;
    var low = model.tags[0]; // model.tags[0] is datamin
    var high = model.tags[1]; // model.tags[1] is datamax
    low = parseFloat(low.toPrecision(3)) // 3 sig figs
    high = parseFloat(high.toPrecision(3)) // 3 sig figs
    lower_slider.value = low;
    lower_slider.set('step', (high-low)/50);
    cm.low = low;
    upper_slider.value = high;
    upper_slider.set('step', (high-low)/50);
    cm.high = high;
    im.glyph.color_mapper.low = low;
    im.glyph.color_mapper.high = high;
    lower_input.value = low.toString();
    upper_input.value = high.toString();
    lower_slider.start = low;
    lower_slider.end = high;
    upper_slider.start = low;
    upper_slider.end = high;
    model.trigger('change')
    cb_obj.trigger('change')
""")

In [8]:
from bokeh.models import Button, Slider, TextInput

# Make the widgets
reset_button = Button(label='Reset', callback = callback_reset_js)
lower_slider = Slider(start=datamin, end=datamax, value=datamin, 
                      step=(datamax-datamin)/50, title="Lower lim", callback=callback_l)
lower_slider.width=100

upper_slider = Slider(start=datamin, end=datamax, value=datamax, 
                      step=(datamax-datamin)/50, title="Upper lim", callback=callback_u)
upper_slider.width=100

lower_input = TextInput(callback=callback_lt, value = str(datamin), width=50)
upper_input = TextInput(callback=callback_ut, value = str(datamax), width=50)

# add all of these widgets as arguments to the callback functions
for callback in ['l', 'u', 'lt', 'ut', 'reset_js']:
    for widget in ['lower_slider', 'upper_slider','lower_input','upper_input', 'reset_button']:
        exec('callback_%s.args["%s"] = %s' %(callback, widget, widget))

In [9]:
from bokeh.layouts import widgetbox, row
# add a widgetbox to the layout
wb = widgetbox([upper_slider, upper_input, lower_slider, lower_input, reset_button], width=100, sizing_mode = 'stretch_both')
layout = row(fig, wb)

In [10]:
show(layout)